# Ejemplos de programación genética
---

Tutorial completo: https://deap.readthedocs.io/en/master/tutorials/advanced/gp.html


## Funciones básicas

In [1]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

import operator
import math

Debemos establecer un conjunto de primitivas. Es decir, las operaciones que constituirán nuestro programa.

In [2]:
pset = gp.PrimitiveSet("main", 2)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addTerminal(3)

Nuestra función de principal tiene dos entradas, las renombraremos como $x$ e $y$.

In [3]:
pset.renameArguments(ARG0="x")
pset.renameArguments(ARG1="y")

Crearemos nuestros individuos mediante el método **full**. Es decir, generaremos el árbol con una cierta profundidad entre *min_* y *max_*. A partir de la expresión *expr* construiremos el árbol *tree* mediante la primitiva <code>gp.PrimitiveTree</code>.

In [4]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)

Veamos qué expresión tenemos:

In [5]:
print(tree)

add(max(3, 3), max(3, x))


Esto no deja de ser una cadena de caracteres. Para poder computar con ella necesitamos compilarla.

In [6]:
f = gp.compile(tree, pset)

Ahora sí podemos obtener resultados cuando introducimos dos parámetros como entrada.

In [7]:
f(7,3)

10

In [8]:
nodes, edges, labels = gp.graph(expr)

### Graphviz Section ###
import pygraphviz as pgv

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("tree.pdf")

<img src="imgs/tree.JPG" width=70%>

## Ejemplo

In [9]:
#    This file is part of EAP.
#
#    EAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    EAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with EAP. If not, see <http://www.gnu.org/licenses/>.

import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof


p_, l_, h_ = main()

   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max  	min     	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	1.78879	0  	30.34	0.450825	300   	2.67896	3.54667	0  	7  	2  	300   	1.49482
1  	166   	1.43254	1  	44.4437	0.183711	166   	3.05668	3.60667	1  	12 	1  	166   	1.77725
2  	166   	2.16879	2  	315.736	0.165572	166   	18.1873	3.55   	2  	9  	1  	166   	1.62506
3  	163   	0.98255	3  	2.9829 	0.165572	163   	0.712666	3.42667	3  	9  	1  	163   	1.45073
4  	153   	0.836017	4  	14.538 	0.165572	153   	0.979399	3.77   	4  	11 	1  	153   	1.64025
5  	158   	0.944635	5  	18.9739	0.165572	158   	1.61614 	3.77667	5  	10 	1  	158   	1.62894
6  	169   	0.885819	6  	14.2181	0.165572	169   	1.00296 	4      	6  	10 	1  	169   	1.87617
7  	167   	0.731332	7  	3.35292	0.165572	167   

In [31]:
h_[0]

In [10]:
print(h_[0])

add(mul(x, sub(x, neg(mul(x, sub(x, neg(mul(x, x))))))), x)


In [11]:
nodes, edges, labels = gp.graph(h_[0])

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("result.pdf")

<img src="imgs/resultGP.JPG" width=40%>